[TEMPLATE] — Review and adapt

🧠 Install Semantic Kernel

To use Microsoft’s Semantic Kernel for orchestrating AI agents, planners, and connectors, install the latest version from PyPI using pip.
Run the following command in a code cell below:

In [ ]:
# TEMPLATE: review/adapt this cell for your project
pip install -U semantic-kernel

[TEMPLATE] — Review and adapt

💡 Note: Removing pydrive2

If you see dependency warnings about cryptography or pyOpenSSL when installing Azure packages, they usually come from pydrive2, which Colab preinstalls for legacy Google Drive APIs.
You don’t actually need pydrive2 for Drive access anymore — Colab’s built-in drive.mount() handles everything.

To remove it safely and avoid version conflicts, run:


In [ ]:
# TEMPLATE: review/adapt this cell for your project
!pip -q uninstall -y pydrive2

[TEMPLATE] — Review and adapt

🤖 Connect to Azure OpenAI in Colab

The following code shows how to connect Google Colab to your Azure OpenAI resource using the official openai Python package.

Import the AzureOpenAI client class.

Initialize it with your Azure OpenAI endpoint, API key, and API version.

Use your deployment name (for example, gpt-35-turbo) as the model parameter when sending chat requests.

Print the assistant’s response.

⚠️ Security Note: Never share your real API key in public notebooks or commits.
Instead, store it in an environment variable (for example,
os.environ["AZURE_OPENAI_API_KEY"]) and reference it securely in your code.

Run the next code cell to send a test message to your Azure OpenAI deployment and confirm the connection from Colab.



In [ ]:
# TEMPLATE: review/adapt this cell for your project
from openai import AzureOpenAI

# Initialize client
client = AzureOpenAI(
    api_key="INSERT_KEY",
    azure_endpoint="INSERT_ENDPOINT}",
    api_version="2024-10-21"
)

# Correct model name = deployment name (not 'gpt-4' etc.)
response = client.chat.completions.create(
    model="gpt-35-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "Hello from Colab and Azure OpenAI!"}
    ]
)

print(response.choices[0].message.content)


[TEMPLATE] — Review and adapt

🧠 Installing Microsoft Agentic Framework Dependencies

Before running Azure OpenAI or the Microsoft Agent Framework in Colab, you’ll need to install the latest compatible SDKs and libraries.
This command ensures that Semantic Kernel, Azure AI Agents, and OpenAI are all properly aligned to avoid version conflicts.

This installs:

🧩 semantic-kernel – orchestrates AI workflows and prompt pipelines

🔐 azure-identity – handles secure authentication with Azure services

☁️ azure-ai-projects – project management and AI service integration

🤖 azure-ai-agents – enables creation of tool-using and multi-agent systems

💬 openai – communicates with Azure OpenAI endpoints for completions and chat

✅ Tip: Run this cell once per Colab session before importing any Azure or Semantic Kernel packages.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
!pip -q install -U "semantic-kernel==1.37.1" "azure-identity>=1.25.1" "azure-ai-projects==1.1.0b4" "azure-ai-agents==1.2.0b5" "openai>=1.98"


[TEMPLATE] — Review and adapt

📂 Work with Google Drive in Colab

Use this cell to mount your Google Drive, inspect files, create a project folder, and read/write a CSV. It avoids pydrive2 by using Colab’s native mount.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)   # follow the auth link

# 2) Your files live under /content/drive/MyDrive
import os, pathlib, pandas as pd

base = pathlib.Path("/content/drive/MyDrive")
print("Root exists?", base.exists())
print("Sample listing:", os.listdir(base)[:10])

# 3) Create a folder and write a file
proj = base / "azure_agentic_demo"
proj.mkdir(exist_ok=True)
(proj / "hello.txt").write_text("Hello from Colab!")

# 4) Save & read a CSV
df = pd.DataFrame({"a":[1,2,3], "b":["x","y","z"]})
csv_path = proj / "sample.csv"
df.to_csv(csv_path, index=False)
print("Saved:", csv_path)

pd.read_csv(csv_path).head()


[TEMPLATE] — Review and adapt

🔑 Configure Azure OpenAI Environment Variables

Set up your Azure credentials so Colab can authenticate directly with your Azure OpenAI resource.
Replace the sample API key and endpoint below with your own values from the Azure Portal (Keys and Endpoint section).

In [ ]:
# TEMPLATE: review/adapt this cell for your project
import os
os.environ["AZURE_OPENAI_ENDPOINT"]   = "https://4th-openai-resource.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"]    = "XXXXX"   # from Keys & Endpoints
os.environ["AZURE_OPENAI_API_VERSION"]= "2024-10-21"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-35-turbo"    # your deployment name


[TEMPLATE] — Review and adapt

✅ Quick sanity check: call your Azure OpenAI deployment

Run this cell to verify your environment variables and deployment are working. It should print a short greeting; if anything’s misconfigured, you’ll see a clear error.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
from openai import AzureOpenAI, OpenAIError
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)

try:
    r = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
        messages=[{"role":"user","content":"Say hi in one short line."}]
    )
    print(r.choices[0].message.content)
except OpenAIError as e:
    print("ERROR:", e)

[TEMPLATE] — Review and adapt

🧪 Tool-calling demo: simple agent with a timezone tool

This cell shows a minimal agent loop using Azure OpenAI tool-calling. The model decides when to call the get_timezone tool, you run the tool in Python, then feed the result back for a final answer.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
import json, datetime as dt

def get_timezone(city:str)->str:
    # stub tool; replace with real logic as needed
    return f"Timezone for {city}: US/Eastern (stub) @ {dt.datetime.now().isoformat(timespec='seconds')}"

tools = [{
    "type":"function",
    "function":{
        "name":"get_timezone",
        "description":"Return a timezone for a US city",
        "parameters":{"type":"object",
                      "properties":{"city":{"type":"string"}},
                      "required":["city"]},
    },
}]

messages=[{"role":"system","content":"You are concise and call tools when helpful."},
          {"role":"user","content":"What timezone is Tallahassee in?"}]

resp = client.chat.completions.create(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
msg = resp.choices[0].message
messages.append(msg)

if msg.tool_calls:
    for call in msg.tool_calls:
        args = json.loads(call.function.arguments or "{}")
        result = get_timezone(**args)
        messages.append({
            "role":"tool",
            "tool_call_id":call.id,
            "name":call.function.name,
            "content": result
        })
    final = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
        messages=messages
    )
    print(final.choices[0].message.content)
else:
    print(msg.content)


[TEMPLATE] — Review and adapt

💾 Save conversation to Google Drive (normalize SDK objects → JSON)

This cell converts any SDK message objects (like ChatCompletionMessage) into plain dicts so they’re JSON-serializable, then writes the transcript to Drive.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
# Normalize OpenAI message objects -> plain dicts, then save to Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pathlib, json

def normalize_messages(msgs):
    out = []
    for m in msgs:
        if isinstance(m, dict):
            out.append(m)
            continue
        item = {"role": getattr(m, "role", None), "content": getattr(m, "content", None)}
        # include tool calls if present
        tcs = getattr(m, "tool_calls", None)
        if tcs:
            item["tool_calls"] = [
                {
                    "id": tc.id,
                    "type": "function",
                    "function": {
                        "name": getattr(tc.function, "name", None),
                        "arguments": getattr(tc.function, "arguments", None),
                    },
                }
                for tc in tcs
            ]
        # include name if present (e.g., tool messages)
        name = getattr(m, "name", None)
        if name: item["name"] = name
        out.append(item)
    return out

log_dir = pathlib.Path("/content/drive/MyDrive/azure_agentic_demo")
log_dir.mkdir(exist_ok=True)
normalized = normalize_messages(messages)
path = log_dir / "conversation.json"
path.write_text(json.dumps(normalized, indent=2))
print(f"Saved {path}")



[TEMPLATE] — Review and adapt

⚙️ Install Microsoft Agent Framework (Preview)

Run this cell to install the Microsoft Agent Framework (pre-release version) into your Colab environment:

In [ ]:
# TEMPLATE: review/adapt this cell for your project
pip install agent-framework --pre

[TEMPLATE] — Review and adapt



In [ ]:
# TEMPLATE: review/adapt this cell for your project
# === RAG over Drive files (PDF/DOCX/TXT/MD) => embed (local) => retrieve => answer with Azure OpenAI ===
# Turnkey single cell for Google Colab

# --- Install deps (idempotent) ---
import sys, subprocess, importlib
def pip_install(pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-U", *pkgs], check=True)

for mod, pkgs in {
    "faiss": ["faiss-cpu"],
    "sentence_transformers": ["sentence-transformers"],
    "pypdf": ["pypdf"],
    "docx2txt": ["docx2txt"],
}.items():
    try:
        importlib.import_module(mod)
    except Exception:
        pip_install(pkgs)

# --- Imports ---
from google.colab import drive
from pypdf import PdfReader
import docx2txt
import os, pathlib, re, json, hashlib, pickle
import numpy as np, faiss
from sentence_transformers import SentenceTransformer
from openai import AzureOpenAI

# --- Mount Drive ---
drive.mount("/content/drive", force_remount=True)

# ========================= CONFIG =========================
# Put your docs into this folder in Drive:
DATA_DIR = pathlib.Path("/content/drive/MyDrive")
DATA_DIR.mkdir(exist_ok=True)

# Where to save FAISS and metadata:
INDEX_DIR = pathlib.Path("/content/drive/MyDrive/RAG_index")
INDEX_DIR.mkdir(exist_ok=True)

# Ask your question here:
QUESTION = "Summarize the key policies and dates from the syllabus."
TOP_K = 5
# =========================================================

print("Place files in:", DATA_DIR)
print("Found files:", [p.name for p in DATA_DIR.iterdir() if p.suffix.lower() in [".pdf",".docx",".txt",".md"]])

# --- Utilities: load & chunk ---
def read_file(path: pathlib.Path) -> str:
    suf = path.suffix.lower()
    if suf == ".pdf":
        text = []
        with open(path, "rb") as f:
            reader = PdfReader(f)
            for pg in reader.pages:
                text.append(pg.extract_text() or "")
        return "\n".join(text)
    if suf == ".docx":
        return docx2txt.process(str(path)) or ""
    if suf in [".txt", ".md"]:
        return path.read_text(errors="ignore")
    return ""

def clean_text(s: str) -> str:
    s = s.replace("\x00", " ")
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def chunk_text(text: str, max_words=300, overlap=60):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+max_words]
        if not chunk: break
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks

def file_digest(path: pathlib.Path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for b in iter(lambda: f.read(1 << 20), b""):
            h.update(b)
    return h.hexdigest()

# --- Gather docs -> chunks + metadata ---
docs = []
file_hashes = {}
for p in sorted(DATA_DIR.iterdir()):
    if p.suffix.lower() not in [".pdf", ".docx", ".txt", ".md"]:
        continue
    raw = read_file(p)
    txt = clean_text(raw)
    if not txt.strip():
        continue
    fh = file_digest(p)
    file_hashes[p.name] = fh
    for j, ch in enumerate(chunk_text(txt, max_words=300, overlap=60)):
        docs.append({"doc": p.name, "digest": fh, "chunk_id": j, "text": ch})

if not docs:
    raise SystemExit("No supported documents found. Add PDF/DOCX/TXT/MD files to /MyDrive/RAG_docs and rerun.")

print(f"Prepared {len(docs)} chunks from {len(file_hashes)} files.")

# --- Build or load embeddings + FAISS ---
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(embed_model_name)

index_path = INDEX_DIR / "index.faiss"
meta_path  = INDEX_DIR / "meta.pkl"
hash_path  = INDEX_DIR / "file_hashes.json"

rebuild = True
if index_path.exists() and meta_path.exists() and hash_path.exists():
    try:
        old_hashes = json.loads(hash_path.read_text())
        if old_hashes == file_hashes:
            # Load existing index/meta
            index = faiss.read_index(str(index_path))
            with open(meta_path, "rb") as f:
                meta_docs = pickle.load(f)
            # Sanity: meta_docs should match docs length; if not, rebuild
            if len(meta_docs) == len(docs):
                docs = meta_docs
                rebuild = False
    except Exception:
        pass

if rebuild:
    print("Embedding and indexing…")
    embs = embed_model.encode(
        [d["text"] for d in docs],
        batch_size=64,
        show_progress_bar=True,
        normalize_embeddings=True
    )
    embs = np.asarray(embs, dtype="float32")
    index = faiss.IndexFlatIP(embs.shape[1])  # cosine (with normalized vectors)
    index.add(embs)
    faiss.write_index(index, str(index_path))
    with open(meta_path, "wb") as f:
        pickle.dump(docs, f)
    hash_path.write_text(json.dumps(file_hashes, indent=2))
    print(f"Indexed {len(docs)} chunks.")
else:
    print("Loaded existing index.")

# --- Retriever ---
def retrieve(query: str, k=TOP_K):
    qvec = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.asarray(qvec, dtype="float32"), k)
    results = []
    for rank, i in enumerate(I[0]):
        if i == -1: continue
        m = docs[i]
        results.append({
            "rank": rank + 1,
            "score": float(D[0][rank]),
            "doc": m["doc"],
            "chunk_id": m["chunk_id"],
            "text": m["text"]
        })
    return results

def make_context(snippets):
    blocks = []
    for s in snippets:
        header = f"[{s['doc']} #chunk:{s['chunk_id']} score:{s['score']:.3f}]"
        blocks.append(f"{header}\n{s['text']}")
    return "\n\n---\n\n".join(blocks)

# --- Azure OpenAI client (expects env vars already set) ---
missing = [k for k in ["AZURE_OPENAI_ENDPOINT","AZURE_OPENAI_API_KEY","AZURE_OPENAI_API_VERSION","AZURE_OPENAI_DEPLOYMENT"] if not os.getenv(k)]
if missing:
    raise SystemExit(f"Missing env vars: {missing}. Set them before running this cell.")

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

def answer_with_rag(question: str, top_k: int = TOP_K):
    hits = retrieve(question, k=top_k)
    context = make_context(hits)
    prompt = (
        "Use the CONTEXT to answer the QUESTION concisely. "
        "If the answer is not in the context, say you don't know.\n\n"
        f"QUESTION:\n{question}\n\n"
        f"CONTEXT:\n{context}\n\n"
        "Return citations like [doc #chunk:X]."
    )
    resp = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT"],  # deployment name, e.g., gpt-35-turbo
        messages=[
            {"role":"system","content":"You are a helpful assistant. Always cite sources as [doc #chunk:X]."},
            {"role":"user","content": prompt}
        ],
        temperature=0.2,
    )
    return resp.choices[0].message.content, hits

# --- Ask the question ---
print("\n=== QUESTION ===")
print(QUESTION)
answer, sources = answer_with_rag(QUESTION, top_k=TOP_K)

print("\n=== ANSWER ===")
print(answer)

print("\n=== SOURCES ===")
for s in sources:
    print(f"{s['rank']:>2}. {s['doc']}  #chunk:{s['chunk_id']}  score:{s['score']:.3f}")
